In [38]:
import re
import json
import requests
#hash this as we'll be using it a lot
def get_page(url,params={}):
    return session_requests.get(
        url,
        params=params,
        headers = dict(referer=url))

session_requests = requests.session()
login_url = 'https://www.vinted.fr/member/general/login?ref_url='
tt = session_requests.get(login_url).content.decode('utf-8')
pp = re.compile('name="csrf-token" content="(.*?)"')
mm = pp.search(tt)

from vinted_creds import creds
payload = creds
payload["authenticity_token"] =  mm.group(1)

result = session_requests.post( 'https://www.vinted.fr/member/general/login.json',
                               json=json.dumps(payload),
                               headers = dict(referer=login_url)
                              )

In [39]:
result

<Response [422]>

In [5]:
url = 'https://www.vinted.fr/api/v2/users/20263980/items'

stuff = []
for k in range(1,7):
    print(k)
    r = get_page(url, 
                params={'page': k, 'per_page': 48})
    stuff.append( r.json())

1
2
3
4
5
6


you should really get the max_page from in the first json batch
['pagination']

In [6]:
stuff = [x['items'] for x in stuff]
import itertools
df = list(itertools.chain.from_iterable(stuff))

In [8]:
its = [ (x['view_count'], x['title'] + x['description']) for x in df ]
its = [ (x['view_count'], x['title'] ) for x in df ]
sum( [ float(x['price_numeric']) for x in df ] )

961.5999999999999

In [9]:
from datetime import datetime
def str2date(ss):
    date = ss.split('+')[0]
    return  datetime.strptime(date, '%Y-%m-%dT%H:%M:%S')

def date2offset(ss):
    start = str2date('2018-12-02T15:53:11+01:00')
    
    return  (datetime.now() - str2date(ss) ).days

xx = [ (x['view_count'], 
        date2offset(x['created_at_ts']),
        x['title'] 
        ) for x in df ]

In [11]:
xx.sort(reverse=True)
xx[:20]

[(92, 1, 'pull ralph lauren femme bleu marine XS '),
 (58, 26, 'veste petit bateau marine fille '),
 (56, 44, 'manteau bleu marine petit bateau fille laine (78%)'),
 (48, 51, 'gants nike femme'),
 (45, 26, 'polo lacoste bleu marine femme '),
 (45, 20, 'basket nike femme (dual fusion X) 38'),
 (45, 20, 'Chaussures Nike Free 5.0 femme 38.5'),
 (42, 10, 'Super bonne paye '),
 (41, 30, 'tee shirt superdry bleu femme '),
 (41, 11, 'Kawaii le livre 100% Japon '),
 (40, 11, 'Mirogolo '),
 (32, 10, 'polo lacoste rose framboise femme 34'),
 (29, 400, 'Veste Jennifer'),
 (28, 22, '211 idées pour devenir un garçon génial '),
 (28, 11, 'Les énigmes de sherlock holmes'),
 (27, 29, 'Manteau ooxoo garçon marine en laine '),
 (26, 44, 'Gants répétons fille rose poudré'),
 (24, 11, 'Les goûters philo'),
 (24, 7, 'Boule et bill - Sos Planète '),
 (23, 22, 'Nos étoiles contraires - John Green')]

In [13]:
with open('classement.txt','w') as fp:
    fp.write( '\n'.join(['%d %d %s'%x for x in xx]) )

In [59]:
r = get_page('https://www.vinted.fr/member/general/followers/20263980')
px = re.compile('membres/(\d+)-(.*?)"')
friends = list( set(px.findall(r.content.decode('utf-8'))) )

In [60]:
friends

[('24794428', 'mariehvz'),
 ('15350990', 'evaamw'),
 ('13862386', 'simojo10'),
 ('12651286', 'locean21'),
 ('31290234', 'bibou0087'),
 ('16147769', 'iannelaure'),
 ('12271904', 'smarchetti'),
 ('32135520', 'bertrandclavel'),
 ('32147659', 'michelinjeanmarc'),
 ('31678660', 'brisbane63'),
 ('28779534', 'mthiollier'),
 ('24794404', 'ceeliaa19'),
 ('20263980', 'erinmcsh'),
 ('10559472', 'adeleonillon'),
 ('15226661', 'karimusnebili'),
 ('26278824', 'colinegvs'),
 ('32423957', 'paul11040'),
 ('32032265', 'manon-rchr38'),
 ('32359937', 'marie-gabvort')]

In [18]:
with open('xx.html','wb') as fp:
    fp.write( r.content )

In [63]:
import time

keys = [ 'login', 'gender', 'city',
'item_count', 'given_item_count', 
'created_at', 'last_loged_on_ts' ]

data = []

for id_,name  in friends:
    print(name)
    r = get_page('https://www.vinted.fr/api/v2/users/%s/'%id_)
    xx = r.json()['user']
    data.append([ xx[k] for k in keys ])
    time.sleep(3)
    
    

mariehvz
evaamw
simojo10
locean21
bibou0087
iannelaure
smarchetti
bertrandclavel
michelinjeanmarc
brisbane63
mthiollier
ceeliaa19
erinmcsh
adeleonillon
karimusnebili
colinegvs
paul11040
manon-rchr38
marie-gabvort


In [74]:
cols = dict( zip( keys, list(zip(*data)) ) )

for t in ['created_at', 'last_loged_on_ts' ]:
    cols[t] = [x.split('T')[0] for x in cols [t] ]

In [69]:
import pandas as pd

In [75]:
pd.DataFrame(cols)

,login,gender,city,item_count,given_item_count,created_at,last_loged_on_ts
0,marie.hvz,F,Grenoble,2,5,2019-05-05,2020-01-07
1,evaa.mw,F,Échirolles,38,19,2018-01-29,2020-01-07
2,simojo10,F,,20,16,2017-09-18,2020-01-07
3,locean21,F,Versailles,29,45,2017-04-08,2020-01-06
4,bibou0087,F,Béziers,418,19,2019-11-13,2020-01-07
5,i.annelaure,M,Grenoble,0,0,2018-03-20,2019-12-08
6,smarchetti,F,,26,4,2017-02-14,2020-01-07
7,bertrand_clavel,M,Grenoble,47,31,2019-12-08,2020-01-07
8,michelin.jeanmarc,,Grenoble,0,0,2019-12-08,2019-12-08
9,brisbane63,F,Marcellaz,17,7,2019-11-23,2020-01-07
